In [46]:
import glob
import math

import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import cv2

import aruco_utils

In [ ]:
px.imshow(np.load("data/data/chessboard.npy")[..., ::-1]).show()

In [ ]:
px.imshow(np.load("data/photo_moving.npy")).show()

In [47]:
images = {
    int(path[24:-4]): np.load(path).reshape((180, 320, 3))
    for path in glob.glob("data/data/photo_new_foc_*.npy")
}
print(len(images))
print(images.keys())

8
dict_keys([2000, 4000, 2500, 3000, 500, 1000, 1500, 3500])


In [50]:
# canny_images = [
#     cv2.Canny(cv2.blur(img, (3, 3)), 10, 70, apertureSize=3) for img in images.values()
# ]
px.imshow(np.array(list(images.values())), animation_frame=0, width=5_00).show()

In [ ]:
px.imshow(images[4600]).show()

In [ ]:
%timeit cv2.cvtColor(images[4600], cv2.COLOR_BGR2GRAY)

In [ ]:
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
arucoParams = cv2.aruco.DetectorParameters()
arucoParams.minDistanceToBorder = 0
(corners, ids, rejected) = cv2.aruco.detectMarkers(
    images[1000][::1, ::1], arucoDict, parameters=arucoParams
)
px.imshow(
    cv2.aruco.drawDetectedMarkers(images[1000][::1, ::1, ::-1].copy(), corners, ids)
).show()

In [ ]:
# 14.3, 14.4 cm shape
marker_size_mm = 144


def guess_focal(img: np.array, dist_mm: int, size_mm: int = marker_size_mm):
    downsample = 1

    # img = cv2.cvtColor(img[::downsample, ::downsample], cv2.COLOR_BGR2GRAY)
    check = cv2.aruco.detectMarkers(img, arucoDict, parameters=arucoParams)[0]
    if not check:
        print(f"Warning: distance {dist_mm} can't be identified")
        return None
    cs = check[0][0]
    pixel_size = np.mean(
        [
            np.sqrt(np.square((cs[0] - cs[3]) * downsample).sum()),
            np.sqrt(np.square((cs[1] - cs[2]) * downsample).sum()),
        ]
    )
    return pixel_size * dist_mm / size_mm


%timeit guess_focal(images[4000], dist_mm=4000)

In [ ]:
cv2.aruco.detectMarkers(images[1000], arucoDict, parameters=arucoParams)

In [ ]:
aruco_utils.sample_markers(images[1000])

In [ ]:
focals = []
for dist, im in images.items():
    f = guess_focal(im, dist)
    if f is None:
        continue
    focals.append(f)
    print(f"{dist: >7} & {focals[-1]: >10.2f} \\\\")
avg_focal = np.mean(focals)
print(f"\nAverage: {avg_focal}")

In [ ]:
print(focals)
np.std(focals)

In [ ]:
test_cam_mat = np.array(
    [
        [avg_focal, 0, 0],
        [0, avg_focal, 0],
        [0, 0, 1],
    ]
)

In [112]:
num = 1_500
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
arucoParams = cv2.aruco.DetectorParameters()
arucoParams.minDistanceToBorder = 0
(corners, ids, rejected) = cv2.aruco.detectMarkers(
    images[num], arucoDict, parameters=arucoParams
)
px.imshow(
    cv2.aruco.drawDetectedMarkers(images[num][..., ::-1].copy(), corners, ids)
).show()

In [113]:
rot = np.empty((3, 3))
cv2.Rodrigues(aruco_utils.sample_markers(images[num])[0].r_vec, dst=rot)
a = cv2.RQDecomp3x3(src=rot)[0][1]
a
# np.degrees(np.arctan2(y_rot[0, 0], y_rot[0, 2]))

-26.366074561401682

In [119]:
np.cos(-np.radians(30))

0.8660254037844387

In [ ]:
_object_points = np.array(
    [
        [-marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, -marker_size_mm / 2, 0],
        [-marker_size_mm / 2, -marker_size_mm / 2, 0],
    ]
)


def estimate_pose(cn, cam_internal=test_cam_mat, obj_points=_object_points):
    ret, r_vec, t_vec = cv2.solvePnP(
        obj_points,
        imagePoints=cn,
        cameraMatrix=cam_internal,
        distCoeffs=np.zeros((4,)),
        flags=cv2.SOLVEPNP_IPPE_SQUARE,
    )
    assert ret, "Failed to solve PnP for pose estimation!"
    r_mat = np.empty((3, 3))
    cv2.Rodrigues(r_vec, dst=r_mat)
    return r_mat, t_vec


estimate_pose(corners[0][0])

In [ ]:
cam_test = np.array([np.load(f"data/cam_rotate_{nth}.npy") for nth in range(20)])
len(cam_test)

In [ ]:
(list(images.values()))[0].shape

In [ ]:
px.imshow(np.array(list(images.values())), animation_frame=0).show()